# BoardGameGeek.com User Reviews

This notebook processes the reviews for the top 250 boardgames, sorted by number of votes received, from the website BoardGameGeek.com. Each step in the process tries to clean up the data before storing in a .json file at the end

These are the steps:
1. Download [the latest zip file](https://boardgamegeek.com/data_dumps/bg_ranks) containing the CSV with all games and summary rating data.
    - This is a manual step and requires the user to be logged into the website
    - After downloading the ZIP, extract the included CSV into the same directory as this notebook
2. Filter list to those with at least 1000 votes and sort by vote count in descending order
3. For each game, use the XML API to retrieve the comments
    - Paginate through each result set as needed
    - Remove unnecessary tags
    - Ignore blank or non-english comments
4. Store data in a .json file

Note that the BoardGameGeek.com [XML API terms](https://boardgamegeek.com/wiki/page/XML_API_Terms_of_Use) state the following:
<blockquote style="font-size:smaller">Your access to the Extensible Markup Language ("XML") Application Programming Interface ("API") provided by BoardGameGeek, LLC ("BGG") and your use of the data provided by the BGG XML API, is pursuant to the Terms of Service generally applicable to all features of BGG, which are available at http://www.boardgamegeek.com/terms and incorporated herein by reference ("TOS"), and any modifications thereto. Capitalized words used herein without definition shall have the meaning ascribed to them in the TOS.<br/>
In addition, due to the particular features of the BGG XML API, by accessing and using information from the BGG XML API, you agree to be bound by the following additional terms.</br>
BGG grants you a worldwide, non-exclusive, royalty-free license to reproduce and display the data available through the BGG XML API, including User Submissions, solely for strictly non-commercial purposes and solely as permitted by the XML API provided by BGG. BGG shall have the right to terminate this license at any time and for any reason in its sole discretion.</br>
If you would like to license the XML API for commercial use, please contact Daniel Karp.</br>
You may not modify the data, including User Submissions, retrieved through the BGG XML API in any way. In all uses of the BGG XML API, you shall credit BoardGameGeek by name as the source of the data.</blockquote>

### Non default libraries used
In case your environment does not have these libraries, execute the following:
`pip install pandas pyarrow lxml requests langdetect`

### Import libraries used in the notebook

In [ ]:
import pandas as pd
import requests #for web requests
from lxml import etree #used to parse the XML returned by the API
from lxml import html #used to parse the HTML
import time #for the wait times
import datetime #for the timestamps
import random #we wait a random number of seconds
from langdetect import detect #library to detect the language a text is in
from langdetect import LangDetectException #for exception handling when using the langdetect library
import re #we're using regular expressions to clean up the text
import zipfile #for the zip file
import os #for file handling
import shutil #to get shell info
import sys #system interaction
from typing import Dict, Union
import json

### Define global variables

In [ ]:
baseURL = 'https://boardgamegeek.com/xmlapi/boardgame/'
all_reviews = []
canProcess = False #flag used to determine if we can start processing or not
checkThreshold = 100000 #threshold for when to flush to disk
outputFile = f".\\boardgamegeek-{datetime.datetime.now().strftime('%Y%m%d')}.json"
gamesSelection = 250 #how many boardgames to select

### Helper functions
Define all the functions that are used in support of the main sections of the code.

* **clearLine()**: Determine the dimensions of the terminal we're running in and clear out the line. Used in conjunction with the `logMessage()` function.
* **logMessage()**: Prints a log message to the terminal with a `\r` and a timestamp, useful for progress messages.
* **checkNumericRange()**: Check if a given value is numeric and within a certain range.
* **removeMarkup()**: Cleans the given text of any markup such as `[url]` or straight HTTP/S URLs.
* **flushBuffer()**: Flushes the `all_reviews` buffer to disk. Though it doesn't do anything for mempry management (a todo), it does save the data regularly to disk just in case of a system crash.

In [ ]:
def clearLine():
    columns, _ = shutil.get_terminal_size(fallback=(80,20)) #get the terminal size retaining just the column count, ignoring the rows
    sys.stdout.write('\r' + ' ' * columns * 2 + '\r') #reset to the beginning of the line and print enough spaces to clear out the line. We're multiplying by 2 to cover the line
    sys.stdout.flush() #flush the buffer

def logMessage(msg: str):
    clearLine() #clear the whole line
    ts = datetime.datetime.now() #get the current timestamp
    print(f"\r{ts}\t{msg}",end='\r',flush=True) #print the message

def checkNumericRange(val: str, min: float, max: float) -> bool:
    try: #if the try/catch block fails, it wasn't a number
        num = float(val) #convert the number to a float
        if min<=num<=max:
            return True
        else:
            return False
    except Exception:
        return False

def removeMarkup(txt: str) -> str:
    cleanText = txt #initialize
    patterns = {r'\[url=.*?\].*?\[/url\]',      #pattern: [url=]...[/url]
                r'\[thing=.*?\].*?\[/thing\]',  #pattern: [thing=]...[/thing]
                r'\[([^\[\]/]+)\]\[/\1\]',      #remove any empty tags eg. [tag][/tag]
                r'https?://[^\s]+',             #pattern: http://... or https://...
                r'\[/?[^\[\]]+\]'               #KEEP LAST! remove all opening & closing tags
                } 
    try:
        for pattern in patterns: #cycle through the regex patterns
            cleanText = re.sub(pattern,'',cleanText)
    except Exception: #ignore any errors
        pass

    return cleanText

def flushBuffer():
    #todo: add memory management
    #save additional data in case of BSOD to allow for better recovery
    fl = open(outputFile, 'w', encoding='utf-8', newline='\n')

    json.dump(all_reviews, fl, indent=4, ensure_ascii=False)


### Get the review data

This is where we actually interact with the XML API. The [documentation](https://boardgamegeek.com/wiki/page/BGG_XML_API2) isn't the greatest and the resulting XML doesn't have a defined style but, the data is straightforward enough to parse. The key bits of data are in the `<name primary="true">` element for the game name and then the reviews are stored in `<comment>` elements.

The `getData()` function makes successive API calls (only 100 reviews per page are returned), cleans up the data, and returns a `Dict` object containing all the reviews for the given game ID

In [ ]:
def getData(gameID: str) -> Dict[str, Union[str, Dict[str, Union[str, str]]]]:
    
    #make sure we got a parameter
    if gameID == None:
        return None
    
    urlParam = '?comments=1&page='
    gameDataURL = baseURL + str(gameID) + urlParam #construct the URL

    #initialize variables
    pgCount = 1
    reviewData = []
    gameName = ''
    keepGoing = True
    commentTotal = 0
    commentTotalGood = 0
    commentPage = 0
    commentGood = 0
    requestErrorCount = 0

    try:
        while keepGoing: #we stay in the loop until we run out of comments
            logMessage(f"Processing page {pgCount}\tTotal comments so far: {commentPage}\tGood in page: {commentGood}\tTotal good so far: {commentTotalGood}")
            resp = requests.get(gameDataURL+str(pgCount))

            if resp.status_code == 200: #only continue if successful
                try:
                    xmlDoc = etree.fromstring(resp.content)
                    comments = xmlDoc.xpath('//boardgames/boardgame/comment') #get all the <comment> elements
                    if len(comments)>0:
                        commentPage = len(comments)
                        commentTotal += commentPage
                        commentGood = 0

                        logMessage(f"Processing page {pgCount}\tTotal comments so far: {commentTotal}\tTotal good so far: {commentTotalGood}\tComments in page: {commentPage}\tGood in page: {commentGood}")
                        if pgCount == 1:
                            gameName = xmlDoc.xpath('//boardgames/boardgame/name[@primary="true"]/text()') #get the boardgame name

                        for comment in comments: #loop through the comments and extract them
                            dummy = comment.get('rating')
                            if checkNumericRange(dummy, 1, 10): #check if we have a valid rating
                                rtg = dummy
                                txt = comment.text
                                txt = removeMarkup(txt) #clean up the review text
                                try:
                                    if detect(txt)=='en': #we only want text in English
                                        reviewData.append({'rating':rtg,'review_text':txt})
                                        commentGood += 1
                                        commentTotalGood += 1
                                except LangDetectException: #ignore
                                    pass
                            logMessage(f"Processing page {pgCount}\tTotal comments so far: {commentTotal}\tTotal good so far: {commentTotalGood}\tComments in page: {commentPage}\tGood in page: {commentGood}")
                        pgCount += 1
                        time.sleep(random.uniform(5, 10)) #we're being nice and waiting between 5-10 seconds before each page
                    else:
                        #we found no more comments so we can exit the while loop
                        keepGoing = False
                except Exception as e:
                    clearLine()
                    requestErrorCount += 1
                    if requestErrorCount >= 3:
                        keepGoing = False
                        logMessage(f"Error {e} while processing page {pgCount}. Recorded at errors at least 3 times, no longer trying.")
                    else:
                        logMessage(f"Error {e} while processing page {pgCount}.")
            else:
                requestErrorCount += 1 #increment the request error counter
                if requestErrorCount >= 3:
                    #if we've gotten 3 error in requesting the XML via the API, then exit the loop
                    keepGoing = False
        clearLine()
        return {'game_name':gameName, 'reviews': reviewData}
    except Exception as e:
        clearLine()
        print(f"Error: {e}")
        return None



### Start the process

Our very first step is to process the file `boardgames_rangs.csv` containing the summary data for all boardgames. The assumption is that the file is in the same directory as this notebook. If it went well, the `canProcess` flag is set to `True` allowing the continuation.

In [ ]:
df = pd.read_csv('.\\boardgames_ranks.csv') #load the CSV
gameIDs = df[df['usersrated']>=1000].sort_values(by='usersrated', ascending=False) #filer & sort
print(f"Total games in file: {len(df)}\tWith ratings: {len(gameIDs)}\tWe're only using the top {gamesSelection}")
gameIDs = gameIDs.head(gamesSelection)
gameIDs.reset_index(drop=True, inplace=True) #reindex the dataframe
canProcess = True

### Main process
If we can process the data (the `canProcess` flag has been set to `True`), iterate through each game/gameid and process the reviews for it from the API.

There is also a mechanism to restart/continue interrupted runs by using the `reprocess`, `reprocessIndex`, and `reprocessIDs` variables.

In [ ]:
totalRows = len(gameIDs)
reprocess = False #set to True if we are continuing a previously interrupted process
reprocessIndex = 0 #in case of reprocessing, set this to the desired index. NOTE: the index is 0-based
reprocessIDs = [] #set to the IDs wanted to reprocess. NOTE: these are the IDs of the games as used by BoardGameGeek.com
if canProcess:
    #we can process the data!
    totalSaved = 0 #used for stats reporting
    checkCount = 0 #used to check against the threshold for flushing to disk
    ts = datetime.datetime.now()
    print(f"\n{ts}\tStarting to process the reviews\n")
    for index, row in gameIDs.iterrows(): #iterate through each row in the filtered & sorted dataframe
        gameID = row['id']
        gameName = row['name']

        #if we are reprocessing/continuing an interrupted run, then we only want to process the leftovers
        if reprocess and (index< reprocessIndex or (not (gameID in reprocessIDs))):
            continue

        ts = datetime.datetime.now()
        print(f"{ts}\tNow processing {index+1} of {totalRows}\t{gameName} (#{gameID})\tTotal votes: {row['usersrated']}")
        
        gameReviews = getData(gameID) #request the XML from the API

        if gameReviews == None:
            print(f"Error processing {gameName} (#{gameID}), skipping")
            time.sleep(random.uniform(5, 10)) #be nice and wait before requesting the next one
            continue
        
        if len(gameReviews['reviews']) > 0: #only add to the main Dict if we had any valid reviews
            totalSaved += len(gameReviews['reviews'])
            ts = datetime.datetime.now()
            print(f"{ts}\tSaving {len(gameReviews['reviews'])} reviews for {gameName}. Total saved so far: {totalSaved}")
            all_reviews.append(gameReviews)

            checkCount += len(gameReviews['reviews'])
            if checkCount >= checkThreshold:
                flushBuffer()
                checkCount = 0 #reset
        else:
            print(f"Not saving anything for {gameName}")

        time.sleep(random.uniform(5, 10)) #be nice and wait before requesting the next one

### Wrap up
We collected all the data so, now we can save to a .json file for further processing.


In [ ]:
# Save the collected data to a JSON file with UTF-8 encoding
flushBuffer()

print(f"Data has been saved to {outputFile}")